# NFT Trading history
## (v1.2022-02-05)


In [ ]:
#Run this to install plotly for the first time (if needed)
!pip install chart-studio

## Libraries and path

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from functools import reduce
import os 
import math
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.express as px
import chart_studio

In [ ]:
# set path
print(os.getcwd())
os.chdir('CSV files path goes here')  # Provide the new path here

## Read data

In [ ]:
# Read csv files (Total Sales and Average price of NFT for different types of NFT)

# Number of sales datasets
no_sales_collectible = pd.read_csv("total_no_sales_collectible.csv", names = ["Date", "TotalSalesCollectible"], header=0)
no_sales_metaverse = pd.read_csv("total_no_sales_metaverse.csv", names = ["Date", "TotalSalesMetaverse"], header=0)
no_sales_game = pd.read_csv("total_no_sales_game.csv", names = ["Date", "TotalSalesGame"], header=0)
no_sales_art = pd.read_csv("total_no_sales_art.csv", names = ["Date", "TotalSalesArt"], header=0)
no_sales_utility = pd.read_csv("total_no_sales_utility.csv", names = ["Date", "TotalSalesUtility"], header=0)
no_sales_defi = pd.read_csv("total_no_sales_defi.csv", names = ["Date", "TotalSalesDefi"], header=0)

# Average NFT values (USD) datasets
avg_nft_price_collectible = pd.read_csv("avg_nft_val_collectible.csv", names = ["Date", "AvgNftPriceCollectible"], header=0)
avg_nft_price_metaverse = pd.read_csv("avg_nft_val_metaverse.csv", names = ["Date", "AvgNftPriceMetaverse"], header=0)
avg_nft_price_game = pd.read_csv("avg_nft_val_game.csv", names = ["Date", "AvgNftPriceGame"], header=0)
avg_nft_price_utility = pd.read_csv("avg_nft_val_utility.csv", names = ["Date", "AvgNftPriceUtility"], header=0)
avg_nft_price_art = pd.read_csv("avg_nft_val_art.csv", names = ["Date", "AvgNftPriceArt"], header=0)
avg_nft_price_defi = pd.read_csv("avg_nft_val_defi.csv", names = ["Date", "AvgNftPriceDefi"], header=0)

## Data cleaning and engineering

In [ ]:
# set start date to filter on
start_date = '2018-01-01'
end_date = '2022-01-31'
# Join all datastes 
dfs = [no_sales_collectible, no_sales_metaverse, no_sales_game, no_sales_art, no_sales_utility, no_sales_defi,
      avg_nft_price_collectible, avg_nft_price_metaverse, avg_nft_price_game, avg_nft_price_art, avg_nft_price_utility, avg_nft_price_defi]
df_final = reduce(lambda left, right: pd.merge(left, right, on='Date', how = 'outer'), dfs)
# change Date column to datetime
df_final['Date'] = pd.to_datetime(df_final['Date'],format='%Y-%m-%d')
# filter and fill NA value
df_final = df_final[(df_final['Date'] >= start_date) & (df_final['Date'] <= end_date)]
df_final = df_final.fillna(0)


In [ ]:
col_total_sales = ['TotalSalesCollectible','TotalSalesMetaverse','TotalSalesGame','TotalSalesArt','TotalSalesUtility','TotalSalesDefi']
col_avg_price = ['AvgNftPriceCollectible','AvgNftPriceMetaverse','AvgNftPriceGame','AvgNftPriceArt','AvgNftPriceUtility','AvgNftPriceDefi']
# group by month-year and take average over the columns
df_final_grouped_month_sales = df_final.groupby(df_final["Date"].dt.to_period("M"))[col_total_sales].sum()
df_final_grouped_month_avg_price = df_final.groupby(df_final["Date"].dt.to_period("M"))[col_avg_price].mean()
df_final_grouped_month_sales['Date'] = df_final_grouped_month_sales.index.strftime("%Y-%m")
df_final_grouped_month_avg_price['Date'] = df_final_grouped_month_avg_price.index.strftime("%Y-%m") 


## Visualization

In [ ]:
# change the wide format of df_final_grouped_ to long format via melt
df_long_sales = pd.melt(df_final_grouped_month_sales, id_vars=['Date'], 
                  value_vars = col_total_sales)

df_long_avg_price = pd.melt(df_final_grouped_month_avg_price, id_vars=['Date'], 
                  value_vars = col_avg_price)
df_long = pd.concat([df_long_sales, df_long_avg_price], axis=1)
# change column names
df_long.columns = ['Date', 'NFT Type', 'Number of Sales', 'drop_date', 'drop_type', 'Average NFT Value (USD)']
df_long = df_long.drop(['drop_date', 'drop_type'], axis = 1)

# drop unnncessary texts for the purpose of labeling
df_long['NFT Type'] = df_long['NFT Type'].str.replace(r'TotalSales', '')

# Scale factor for better bubble visibility
#df_long["scale"] = df_long["Number of Sales"]**0.4
df_long["scale"] = df_long['Average NFT Value (USD)']




# plotly scatter plot 
fig = px.scatter(df_long, 
                 x = df_long['Date'], 
                 #y = df_long['Average NFT Value (USD)'], 
                 y = df_long['Number of Sales'], 
                 color = df_long['NFT Type'], 
                 size = df_long['scale'],
                 hover_name = df_long['NFT Type'],
                 hover_data=['Date', 'NFT Type', 'Number of Sales', 'Average NFT Value (USD)', ], 
                 opacity=0.5,
                 log_y = True,
                 template = "plotly_dark"               
                )

# print(fig.data[0].hovertemplate)
# Change and update hover texts if needed
fig.update_traces(hovertemplate='<b>%{hovertext}</b><br><br>NFT Type=%{customdata[0]}<br>Date=%{x}<br>Number of Sales=%{y}<br>Average NFT Value (USD)=%{customdata[1]}<extra></extra>') 
fig.update_xaxes(showgrid=True, ticklabelmode="period", dtick="M3", tickformat="%b\n%Y")

fig.update_layout(
    height=700,
    title_text='Number of sales and average price of NFTs (Zoom in, left or double click on legends to filter)',
    xaxis=dict(range=["2017-12-01","2022-01-30"])
)

# Show plot 
fig.show()

# Upload on Plotly Chart Studio for embedding (Details: https://jennifer-banks8585.medium.com/how-to-embed-interactive-plotly-visualizations-on-medium-blogs-710209f93bd)
username='your user name'
api_key='your generated API key'
chart_studio.tools.set_credentials_file(username=username,
                                        api_key=api_key)
py.plot(fig, filename="nft_market_analysis", auto_open = True)